In [ ]:
import os
import subprocess
import sys
import math
from collections import defaultdict
from Bio import Entrez
import time

import pandas as pd
import json
from ete3 import NCBITaxa
from Bio import SeqIO

In [ ]:
sys.path.insert(0, '../utils')
from reference_finder import download_reference_genome, unpack, cat_reference_genome
from alignment import run_minimap2, sort_samfile, samtools_calculate_depth
from summary import alignment_1_summary, alignment_2_summary

In [ ]:
import warnings
warnings.filterwarnings("ignore")

In [ ]:
def get_species_taxid(taxid, ncbi_taxa_db, valid_kingdom={2, 4751, 2157, 10239}):
    lineage = ncbi_taxa_db.get_lineage(taxid)
    if bool(set(lineage) & valid_kingdom):
        taxid2rank_dict = ncbi_taxa_db.get_rank(lineage)
        for lineage_taxid in taxid2rank_dict:
            if taxid2rank_dict[lineage_taxid] == 'species':
                return lineage_taxid
    return None

In [ ]:
def filter_seqscreen_taxonomy(seqscreen_output, min_frac, ncbi_taxa_db, valid_kingdom):
    classification_result_df = pd.read_csv(os.path.join(seqscreen_output, 
                                                        'taxonomic_identification', 
                                                        'taxonomic_assignment', 
                                                        'taxonomic_results.txt'), 
                                           sep='\t')
    total_read_count, _ = classification_result_df.shape
    
    taxid_count_dict = defaultdict(int)
    taxid_species_lookup = dict()
    error_count = 0
    for taxid in classification_result_df['taxid']:
        try:
            taxid = int(taxid)
            try:
                species_taxid = taxid_species_lookup[taxid]
            except KeyError:
                species_taxid = get_species_taxid(taxid, ncbi_taxa_db, valid_kingdom)
                taxid_species_lookup[taxid] = species_taxid

            if species_taxid is not None:
                taxid_count_dict[species_taxid] += 1
        except ValueError:
            error_count += 1

    taxid_queries = []
    for taxid in taxid_count_dict:
        if taxid_count_dict[taxid] >= min_frac * total_read_count:
            taxid_queries.append(taxid)
            
    return taxid_queries

In [ ]:
def prepare_reference_genomes(taxid_queries, output_directory, ncbi_taxa_db):
    working_dir = os.path.join(output_directory, 'ncbi_downloads')
    if not os.path.exists(working_dir):
        os.mkdir(working_dir)
        
    download_result = []
    for taxid in taxid_queries:
        download_result.append(download_reference_genome(taxid, working_dir))
        
    unpack(working_dir, output_directory)
    reference_metadata = pd.DataFrame(download_result,
                                      columns=['Taxonomy ID', 
                                               'Assembly Accession ID', 
                                               'Source Database', 
                                               'Is Representative', 
                                               'Assembly Level', 
                                               'Organism of Assembly',
                                               'Strain',
                                               'Downloaded'])
    
    taxonomy_name = []
    for taxid in reference_metadata['Taxonomy ID']:
        taxonomy_name.append(ncbi_taxa_db.get_taxid_translator([taxid])[taxid])
    reference_metadata['Species'] = taxonomy_name
    
    reference_metadata.to_csv(os.path.join(output_directory, 'reference_metadata.csv'), index=False)
    cat_reference_genome(reference_metadata, output_directory, reference_genome_path=os.path.join(output_directory, 'reference_genomes'))
    
    return reference_metadata

## Load ETE3 NCBITaxa

In [ ]:
database_path = '/home/dbs/SeqScreenDB_21.4'

ete3db = os.path.join(database_path, "reference_inference", "taxa.sqlite")
sequences_db_f = os.path.join(database_path, "bowtie2", "blacklist.seqs.nt.fna")
mapping_f = os.path.join(database_path, "reference_inference", "taxid2seqid.pickle")

ncbi_taxa_db = NCBITaxa(dbfile=ete3db)

In [ ]:
output_directory = "/home/Users/yl181/seqscreen_nano/test"

In [ ]:
input_fasta = '/home/Users/yl181/seqscreen_nano/input_datasets/ERR3152364.downsampled.fasta'
seqscreen_output = "/home/Users/yl181/seqscreen_nano/output_datasets/ZymoBIOMICS.STD.Even.ont.seqscreen"
threads = 40
min_frac = 0.002
min_mapq = 20
min_coverage_score = 0.7

In [ ]:
# ncbi.update_taxonomy_database()

## Filtering Seqscreen Taxonomic Assignment

In [ ]:
taxid_queries = filter_seqscreen_taxonomy(seqscreen_output, 
                                          min_frac=min_frac, 
                                          ncbi_taxa_db=ncbi_taxa_db, 
                                          valid_kingdom={2, 4751, 2157, 10239})

## Fetch Reference Genomes

In [ ]:
reference_metadata = prepare_reference_genomes(taxid_queries, output_directory, ncbi_taxa_db)

In [ ]:
reference_metadata = pd.read_csv(os.path.join(output_directory, 'reference_metadata.csv'))

In [ ]:
downloaded_assemblies = reference_metadata[reference_metadata['Downloaded']]
for assembly_id in downloaded_assemblies['Assembly Accession ID']:
    reference_fasta = os.path.join(output_directory, 'reference_genomes', f'{assembly_id}.fasta')
    run_minimap2(input_fasta, reference_fasta, assembly_id, output_directory, threads=threads)
    sort_samfile(assembly_id, output_directory, min_mapq, threads)
    samtools_calculate_depth(assembly_id, output_directory)

In [ ]:
reference_metadata

In [ ]:
downloaded_assemblies = alignment_1_summary(downloaded_assemblies, output_directory)

In [ ]:
#downloaded_assemblies.sort_values('Coverage Score', ascending=False)

In [ ]:
filtered_assemblies = list(downloaded_assemblies[downloaded_assemblies['Coverage Score'] >= min_coverage_score]['Assembly Accession ID'])

In [ ]:
def merge_reference_fasta(assembly_ids, output_directory):
    merged_fasta = os.path.join(output_directory, 'reference_genomes', f'merged.fasta')
    
    seq_records = []
    for assembly_id in assembly_ids:
        reference_fasta = os.path.join(output_directory, 'reference_genomes', f'{assembly_id}.fasta')

        with open(reference_fasta, "r") as handle:
            for record in SeqIO.parse(handle, "fasta"):   
                seq_records.append(record)
                    
    with open(merged_fasta, "w") as output_handle:
        SeqIO.write(seq_records, output_handle, "fasta")
		
    return merged_fasta

In [ ]:
reference_fasta = merge_reference_fasta(filtered_assemblies, output_directory)
run_minimap2(input_fasta, reference_fasta, 'merged', output_directory, threads=threads)
sort_samfile('merged', output_directory, min_mapq, threads)
samtools_calculate_depth('merged', output_directory)

In [ ]:
downloaded_assemblies = pd.read_csv(os.path.join(output_directory, 'alignment.csv'))

In [ ]:
downloaded_assemblies = alignment_2_summary(downloaded_assemblies, output_directory)

In [ ]:
downloaded_assemblies.sort_values('CS2', ascending=False)